# Facenet recreation
Google facenet recreation

In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import itertools
from matplotlib import image
import glob as glob
from PIL import Image

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
if torch.backends.mps.is_available():
  print("Using the GPU!")
else:
  print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")


PyTorch Version:  2.1.1
Torchvision Version:  0.16.1
Using the GPU!


# Build Dataset from facenet
using instructions from pytorch docs: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
Reading pairs to ensure that muiltiple faces of a single person 


In [2]:
class LabledFacesWild(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        label_dict = []
        with open(annotations_file) as f:
            for line in f:
                print(line)
                name, i1, i2 = line.split('\t')
                label_dict.append({
                    "label": name,
                    "img1": f"{name}/{int(i1):04d}.jpg",
                    "img2": f"{name}/{int(i2):04d}.jpg"
                })
        self.img_labels = pd.DataFrame.from_records(label_dict)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels) * 2

    def __getitem__(self, idx):
        pair_num = idx // 2
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[pair_num, (idx % 2) + 1])
        image = read_image(img_path)
        label = self.img_labels.iloc[pair_num, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [3]:
# actually load data
num_train_pairs = 1100
num_test_pairs = 500
batch_size = 40

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# construct the dataloader
train_data = LabledFacesWild('./pairsDevTrain.txt', './lfw/', transform)
test_data = LabledFacesWild('./pairsDevTest.txt', './lfw/', transform)
train_loader = DataLoader(train_data, batch_size=50, shuffle=False)
test_loader = DataLoader(test_data, batch_size=50, shuffle=False)

# todo figure out actual batching while including groups

Aaron_Peirsol	1	2

Aaron_Peirsol	3	4

Aaron_Sorkin	1	2

Abdel_Nasser_Assidi	1	2

Abdullah	1	3

Abdullah	2	4

Abdullah_al-Attiyah	1	3

Abdullah_al-Attiyah	2	3

Abel_Pacheco	1	4

Abel_Pacheco	3	4

Adam_Sandler	1	3

Adam_Sandler	1	4

Adam_Sandler	2	4

Adam_Scott	1	2

Adolfo_Aguilar_Zinser	1	3

Ahmed_Chalabi	2	4

Ai_Sugiyama	2	4

Aicha_El_Ouafi	1	3

Aicha_El_Ouafi	2	3

Akbar_Hashemi_Rafsanjani	1	3

Akhmed_Zakayev	1	3

Akhmed_Zakayev	2	3

Al_Gore	1	3

Alan_Ball	1	2

Alberto_Fujimori	1	2

Alberto_Ruiz_Gallardon	1	2

Albrecht_Mentz	1	2

Alec_Baldwin	1	2

Alec_Baldwin	1	4

Alec_Baldwin	2	3

Alec_Baldwin	3	4

Alejandro_Toledo	20	38

Alex_Barros	1	2

Alex_Sink	2	3

Alexandra_Stevenson	1	2

Alexandra_Stevenson	1	3

Alexandra_Vodjanikova	1	2

Ali_Abbas	1	2

Ali_Khamenei	1	2

Alice_Fisher	1	2

Alicia_Silverstone	1	2

Alimzhan_Tokhtakhounov	1	2

Alison_Lohman	1	2

Alvaro_Noboa	1	2

Alvaro_Noboa	2	3

Alvaro_Silva_Calderon	3	4

Amanda_Beard	1	2

Amber_Tamblyn	1	2

Amelia_Vega	2	4

Amelia_Vega	5	7

Ame

ValueError: too many values to unpack (expected 3)

# Create Facenet Model
Facenet model https://arxiv.org/pdf/1503.03832.pdf
Google Lenet 22layer: https://arxiv.org/pdf/1409.4842.pdf
input size = 250x250x3
use relu


In [81]:
class Facenet_NN1(nn.Module):
    def __init__(self):
        super().__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        # torch.nn.LocalResponseNorm(size, alpha=0.0001, beta=0.75, k=1.0)
        self.conv1 = nn.Conv2d(3, 64, 7,padding=3, stride=2)
        self.pool1 = nn.MaxPool2d(3, padding=1, stride=2)
        self.rnorm1 = nn.LocalResponseNorm(64)
        
        self.conv2a = nn.Conv2d(64, 64, 1, stride=1)
        self.conv2 = nn.Conv2d(64, 192, 3,padding=1, stride=1)
        self.rnorm2 = nn.LocalResponseNorm(192)
        self.pool2 = nn.MaxPool2d(3, padding=1, stride=2)

        self.conv3a = nn.Conv2d(192, 192, 1, stride=1)
        self.conv3 = nn.Conv2d(192, 384, 3, padding=1, stride=1)
        self.pool3 = nn.MaxPool2d(3, padding=1, stride=2)

        self.conv4a = nn.Conv2d(384, 384, 1, stride=1)
        self.conv4 = nn.Conv2d(384, 256, 3, padding=1, stride=1)

        self.conv5a = nn.Conv2d(256, 256, 1, stride=1)
        self.conv5 = nn.Conv2d(256, 256, 3,padding=1, stride=1)

        self.conv6a = nn.Conv2d(256, 256, 1, stride=1)
        self.conv6 = nn.Conv2d(256, 256, 3,padding=1, stride=1)
        self.pool4 = nn.MaxPool2d(3,padding=1, stride=2)

        #todo what is concat layer?
        self.fc1 = nn.Linear(256*7*7, 128*7*7)
        self.maxout1 = nn.AdaptiveMaxPool2d((32,1))
        self.fc2 = nn.Linear(128 * 32, 128 * 32)
        self.maxout2 = nn.MaxPool2d(2)

        self.fc7128 = nn.Linear(64*16,128)
        
        
        

    def forward(self, x):
        batch_size = x.size(0)
        x1 = self.rnorm1(self.pool1(F.relu(self.conv1(x))))
        x2 = self.pool2(self.rnorm2(F.relu(self.conv2(self.conv2a(x1)))))
        x3 = self.pool3(F.relu(self.conv3(self.conv3a(x2))))
        x4 = F.relu(self.conv4(self.conv4a(x3)))
        x5 = F.relu(self.conv5(self.conv5a(x4)))
        x6 = self.pool4(F.relu(self.conv6(self.conv6a(x5))))
        x6i = torch.flatten(x6, 1)
        x7 = self.fc1(x6i)
        x7i = torch.unflatten(x7, 1, (128, 7, 7))
        x8 = self.maxout1(x7i)
        x8i = torch.flatten(x8, 1)
        x9 = self.fc2(x8i)
        x9i = torch.unflatten(x9, 1, (128, 32))
        x10 = self.maxout2(x9i)
        x10i = torch.flatten(x10, 1)
        x11 = self.fc7128(x10i)
        # normalize the output to a unit vector
        x11 = F.normalize(x11)

        #x7 = self.maxout2(self.fc2(self.maxout1(self.fc1(x6))))
        # ─Conv2d: 1-16                           [1, 256, 6, 6]            590,080
        # ─MaxPool2d: 1-17                        [1, 256, 2, 2]            --
        # return self.fc1(x6)
        return x11
        # return self.fc7128(x7)
        # return x5

In [60]:
# # create inception module
# # inspired by https://www.kaggle.com/code/mohamedmustafa/10-implement-inceptionnet-from-scratch-pytorch
# class ConvBlock(nn.Module):
#     def __init__(self, In_Channels, Out_Channels, Kernel_Size, Stride, Padding):
#         super(ConvBlock, self).__init__()
#         self.Conv = nn.Conv2d(in_channels=In_Channels, out_channels=Out_Channels, kernel_size=Kernel_Size, stride=Stride, padding=Padding)
#         self.Batch_Norm = nn.BatchNorm2d(num_features=Out_Channels)
#         self.Activ_Func = nn.ReLU()
    
#     """
#     Now we'll build the forward function which defines the path to input tensor
#     meaning that we tell the tensor the sequence of layers you're going through 
#     Takecare the name of forward function is sensitive so you have to name forward not any thing else
#     """
#     def forward(self, Tensor_Path):
#         Tensor_Path = self.Conv(Tensor_Path)
#         Tensor_Path = self.Batch_Norm(Tensor_Path)
#         Tensor_Path = self.Activ_Func(Tensor_Path)
        
#         return Tensor_Path

# class InceptionModule(nn.Module):
#     def __init__(self, num1x1, num3x3reduce, num3x3, num5x5reduce, num5x5, ) -> None:
#         super().__init__()


# # NN2
# class Facenet_NN2(nn.Module):
#     def __init__(self):
#         super().__init__()
    
#     def forward(self, x):
#         return x

In [82]:
# print summary
model = Facenet_NN1()
summary(model, input_size=(10, 3, 220, 220), verbose=2)

Layer (type:depth-idx)                   Output Shape              Param #
Facenet_NN1                              [10, 128]                 --
├─Conv2d: 1-1                            [10, 64, 110, 110]        9,472
│    └─weight                                                      ├─9,408
│    └─bias                                                        └─64
├─MaxPool2d: 1-2                         [10, 64, 55, 55]          --
├─LocalResponseNorm: 1-3                 [10, 64, 55, 55]          --
├─Conv2d: 1-4                            [10, 64, 55, 55]          4,160
│    └─weight                                                      ├─4,096
│    └─bias                                                        └─64
├─Conv2d: 1-5                            [10, 192, 55, 55]         110,784
│    └─weight                                                      ├─110,592
│    └─bias                                                        └─192
├─LocalResponseNorm: 1-6                 [10, 192,

Layer (type:depth-idx)                   Output Shape              Param #
Facenet_NN1                              [10, 128]                 --
├─Conv2d: 1-1                            [10, 64, 110, 110]        9,472
│    └─weight                                                      ├─9,408
│    └─bias                                                        └─64
├─MaxPool2d: 1-2                         [10, 64, 55, 55]          --
├─LocalResponseNorm: 1-3                 [10, 64, 55, 55]          --
├─Conv2d: 1-4                            [10, 64, 55, 55]          4,160
│    └─weight                                                      ├─4,096
│    └─bias                                                        └─64
├─Conv2d: 1-5                            [10, 192, 55, 55]         110,784
│    └─weight                                                      ├─110,592
│    └─bias                                                        └─192
├─LocalResponseNorm: 1-6                 [10, 192,

# Train Facenet Model
training data
In all our experiments we train the CNN using Stochastic
Gradient Descent (SGD) with standard backprop [8, 11] and
AdaGrad [5]. In most experiments we start with a learning
rate of 0.05 which we lower to finalize the model. The models are initialized from random, similar to [16], and trained
on a CPU cluster for 1,000 to 2,000 hours. The decrease in
the loss (and increase in accuracy) slows down drastically
after 500h of training, but additional training can still significantly improve performance. The margin α is set to 0.2.

In [ ]:
# todo train

def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for i, batch in enumerate(tqdm_object):
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))

        if i % 100 == 0:
            print('loss:', loss.item() / count)
    return loss_meter


# def valid_epoch(model, valid_loader):
#     loss_meter = AvgMeter()

#     tqdm_object = tqdm(valid_loader, total=len(valid_loader))
#     for batch in tqdm_object:
#         batch = {k: v.to(device) for k, v in batch.items() if k != "caption"}
#         loss = model(batch)

#         count = batch["image"].size(0)
#         loss_meter.update(loss.item(), count)

#         tqdm_object.set_postfix(valid_loss=loss_meter.avg)
#     return loss_meter




model = Facenet_NN1().to(device)
patience = 1
factor = 0.8
epochs = 10

optimizer = torch.optim.Adagrad(model.parameters(), lr=0.05,)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", patience=patience, factor=factor
)
step = "epoch"

best_loss = float('inf')
for epoch in range(epochs):
    print(f"Epoch: {epoch + 1}")
    model.train()
    train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(model, valid_loader)

    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        torch.save(model.state_dict(),  "best.pt")
        print("Saved Best Model!")

    lr_scheduler.step(valid_loss.avg)


# Test Facenet Model
testing

In [ ]:
# todo test